# dynamo db - create table

In [ ]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()
    
print("imported modules.")

## dynamodb client

In [ ]:
client = boto3.client('dynamodb',
  endpoint_url = "http://xanaxprincess.asuscomm.com:8001"
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-access-key"],
  region_name='us-east-2')

## create dynamo table

In [ ]:
dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "https://xanaxprincess.asuscomm.com:8001", \
            region_name='us-east-2')

table = dynamodb.create_table(
    TableName='tweets',
    KeySchema=[
        {
            'AttributeName': 'user',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'created_at',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'test',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'id',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'retweet_count',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'favorite_count',
            'AttributeType': 'N'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

print("Table status:", table.table_status)

## wait for table creation

In [ ]:
session = botocore.session.get_session()
dynamodb = session.create_client('dynamodb', region_name='us-east-2') # low-level client

waiter = dynamodb.get_waiter('table_exists')
waiter.wait(TableName="tweets")